In [2]:
!pip install dash dash-table
!pip install dash-bootstrap-components
!pip install prophet

import dash
import dash_table
import pandas as pd
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc
import plotly.express as px
from google.colab import drive
from prophet import Prophet

# Mount Google Drive
drive.mount('/content/drive')

# Load data from CSV file
data = pd.read_csv('/content/drive/My Drive/datasets/India_Economy_dataset/India_Economic_Growth_Historical_Data.csv')

# Create Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])

# Data Cleaning and Transformation
data.rename(columns={'Unnamed: 0': 'Serial_no'}, inplace=True)
data['year'] = pd.to_numeric(data['year'], errors='coerce')

def clean_gdp(gdp_value):
    if isinstance(gdp_value, str):
        gdp_value = gdp_value.replace('$', '').replace('B', '').replace(',', '').strip()
        try:
            return float(gdp_value)
        except:
            return None
    return None

data['GDP_Clean'] = data['GDP'].apply(clean_gdp)
data['Decade'] = (data['year'] // 10 * 10).astype('Int64')

# Define layout with enhanced features
controls = dbc.Card(
    [
        html.Div(
            [
                dbc.Label("X variable"),
                dcc.Dropdown(
                    id='x-axis',
                    options=[{'label': col, 'value': col} for col in data.columns],
                    value='year', style={'color': 'blue'}
                ),
            ]
        ),
        html.Div(
            [
                dbc.Label("Y variable"),
                dcc.Dropdown(
                    id='y-axis',
                    options=[{'label': col, 'value': col} for col in data.columns],
                    value='GDP_Clean', style={'color': 'blue'}
                ),
            ]
        ),
        html.Div(
            [
                dbc.Label("Color Palette (Z Variable)"),
                dcc.Dropdown(
                    id='z-axis',
                    options=[
                        {'label': 'Rainbow', 'value': 'rainbow'},
                        {'label': 'Warm', 'value': 'warm'},
                        {'label': 'Cool', 'value': 'cool'},
                        {'label': 'Red-Green-Blue', 'value': 'rgb'},
                        {'label': 'Orange-Purple-Cyan', 'value': 'opc'},
                    ],
                    value='rainbow', style={'color': 'blue'}
                ),
            ]
        ),
        html.Div(
            [
                dbc.Label("Graph type"),
                dcc.Dropdown(
                    id='graph-type',
                    options=[
                        {'label': 'Line chart', 'value': 'line'},
                        {'label': 'Bar chart', 'value': 'bar'},
                        {'label': 'Scatter plot', 'value': 'scatter'},
                        {'label': 'Sunburst chart', 'value': 'sunburst'},
                        {'label': 'Treemap chart', 'value': 'treemap'},
                        {'label': 'Pie chart', 'value': 'pie'}
                    ],
                    value='line', style={'color': 'blue'}
                ),
            ]
        ),
        html.Div(
            [
                dbc.Label("Forecast Periods (for Line Chart)"),
                dcc.Input(id="forecast-periods", type="number", value=5, style={'color': 'blue'}),
            ]
        ),
        html.Div(
            [
                dbc.Label("Filter by Year"),
                dcc.RangeSlider(
                    id="year-slider",
                    min=data["year"].min(),
                    max=data["year"].max(),
                    step=1,
                    value=[data["year"].min(), data["year"].max()],
                    marks={str(year): str(year) for year in data["year"].unique()},
                ),
            ]
        ),
    ],
    body=True,
    style={'border-style': 'dotted', 'border-color': 'red'}
)

app.layout = dbc.Container(
    [
        html.H1("India's Economy Growth Dashboard", style={'text-align': 'center'}),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(controls, md=4),
                dbc.Col(dcc.Graph(id='plot', className='plot', style={'margin-top': '25px'}), md=8),
                dbc.Col(
                    dash_table.DataTable(
                        id='data-table',
                        columns=[{'name': col, 'id': col} for col in data.columns],
                        data=data.to_dict('records'),
                        page_size=10,
                        style_table={'overflowX': 'auto'},
                        style_cell={
                            'height': 'auto',
                            'minWidth': '180px', 'width': '180px', 'maxWidth': '180px',
                            'whiteSpace': 'normal'
                        },
                        style_data={'color': 'black'},
                        style_header={'color': 'black'},
                        filter_action="native",
                        sort_action="native",
                        sort_mode="multi",
                    ),
                    md=12
                ),
            ],
            align="center",
        ),
    ],
    fluid=True,
)

# Callback for plot update with forecasting and filtering
@app.callback(
    dash.dependencies.Output('plot', 'figure'),
    [
        dash.dependencies.Input('x-axis', 'value'),
        dash.dependencies.Input('y-axis', 'value'),
        dash.dependencies.Input('graph-type', 'value'),
        dash.dependencies.Input('forecast-periods', 'value'),
        dash.dependencies.Input("year-slider", "value"),
    ]
)
def update_plot_with_forecast(x_col, y_col, graph_type, forecast_periods, year_range):
    filtered_data = data[(data["year"] >= year_range[0]) & (data["year"] <= year_range[1])]

    if graph_type == 'line':
        fig = px.line(filtered_data, x=x_col, y=y_col)
        if x_col == 'year' and y_col in ['GDP_Clean', 'Annual_percent_growth', 'Agriculture_percent_growth', 'Industry_percent_growth', 'Manufacturing_percent_growth', 'Services_percent_growth']:
            df = filtered_data[['year', y_col]].rename(columns={'year': 'ds', y_col: 'y'})
            model = Prophet()
            model.fit(df)
            future = model.make_future_dataframe(periods=forecast_periods, freq='Y')
            forecast = model.predict(future)
            fig.add_scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Forecast', line=dict(color='red'))
            fig.add_scatter(x=forecast['ds'], y=forecast['yhat_lower'], mode='lines', name='Lower Bound', line=dict(color='orange', dash='dash'))
            fig.add_scatter(x=forecast['ds'], y=forecast['yhat_upper'], mode='lines', name='Upper Bound', line=dict(color='orange', dash='dash'))
    elif graph_type == 'bar':
        fig = px.bar(filtered_data, x=x_col, y=y_col)
    elif graph_type == 'scatter':
        fig = px.scatter(filtered_data, x=x_col, y=y_col, size='GDP_Clean', log_x=True, size_max=60)
    elif graph_type == 'sunburst':
        fig = px.sunburst(filtered_data, path=['Decade', 'year'], values='GDP_Clean')
    elif graph_type == 'treemap':
        fig = px.treemap(filtered_data, path=['Decade', 'year'], values='GDP_Clean')
    elif graph_type == 'pie':
        fig = px.pie(filtered_data, values='GDP_Clean', names='year')

    return fig

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<IPython.core.display.Javascript object>